## Exploratory Data Analysis on the Laptop Prices Dataset

### 1.0 Importando as libs

In [0]:
import sys
import os

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [0]:
df = spark.sql("select * from submission")

In [0]:
df.display()

### 2.0 Carregar o dataset

In [0]:
prices_df = pd.read_csv("https://raw.githubusercontent.com/micheldearaujo/datasets/main/laptop_price/laptop_price.csv", sep=',', encoding='latin1')

prices_df.head()

In [0]:
prices_df.shape

In [0]:
prices_df.info()

### 3.0 Explorando os dados

#### 3.1 Verificando os valores únicos de cada coluna
Vamos começar entendendo como estão distribuídas nossas variáveis (numeros vs texto) para descobrir o Feature Engineering necessário

In [0]:
for column in prices_df.columns:
    print(f"Coluna: {column} | Valores únicos: {prices_df[column].nunique()}")
    print(prices_df[column].unique())
    print("-"*100)

#### 3.2 Começamos dropand uma coluna que com certeza não vamos usar devido a alta cardinalidade: Nome do produto

In [0]:
prices_df = prices_df.drop("Product", axis=1)

#### 3.3 Fazendo One Hot Encoding na coluna Company Name

In [0]:
prices_df["Company"].value_counts()

In [0]:
companies_to_agg = ["Razer",
"Mediacom",
"Microsoft",
"Xiaomi",
"Vero",
"Chuwi",
"Google",
'Fujitsu',
'LG',
'Huawei']

In [0]:
prices_df["Company"] = prices_df["Company"].apply(lambda x: "Other" if x in companies_to_agg else x)

In [0]:
# Screen Resolution - A resolução é sempre no final da string, então é fácil
prices_df["ScreenResolution"] = prices_df["ScreenResolution"].str.split(" ").apply(lambda x: x[-1])
prices_df["ScreenWidth"] =  prices_df["ScreenResolution"].str.split("x").apply(lambda x: x[0]).astype("int")
prices_df["ScreenHeight"] =  prices_df["ScreenResolution"].str.split("x").apply(lambda x: x[1]).astype("int")

In [0]:
# CPU é mesma coisa, a frequencia do CPU é sempre a ultima e a marca é a primeira
prices_df["CPU_BRAND"] =  prices_df["Cpu"].str.split(" ").apply(lambda x: x[0])
prices_df["CPU_FREQUENCY"] =  prices_df["Cpu"].str.split(" ").apply(lambda x: x[-1])
prices_df["CPU_FREQUENCY"] =  prices_df["CPU_FREQUENCY"].apply(lambda x: x[:-3]).astype("float")

In [0]:
# RAM
prices_df["Ram"] =  prices_df["Ram"].apply(lambda x: x[:-2]).astype("int")

In [0]:
def convert_to_gb(string):
    number = float(string[:-2])

    if "TB" in string:
        number = number*1024
    
    return number

In [0]:
# Memory
prices_df["Memory_Size"] = prices_df["Memory"].str.split(" ").apply(lambda x: x[0])
prices_df["Memory_Size"] = prices_df["Memory_Size"].apply(lambda x: convert_to_gb(x))
prices_df["Memory_Type"] = prices_df["Memory"].str.split(" ").apply(lambda x: x[1])

In [0]:
# Weight
prices_df["Weight"] = prices_df["Weight"].apply(lambda x: x[:-2]).astype("float")

In [0]:
# GPU
prices_df["GPU_BRAND"] = prices_df["Gpu"].str.split(" ").apply(lambda x: x[0])

In [0]:
# Get Dummies das variaveis categóricas finais
prices_df = prices_df.join(pd.get_dummies(prices_df["Company"], prefix="company", dtype="int"))
prices_df = prices_df.join(pd.get_dummies(prices_df["TypeName"], prefix="typeName", dtype="int"))
prices_df = prices_df.join(pd.get_dummies(prices_df["CPU_BRAND"], prefix="CPU_BRAND", dtype="int"))
prices_df = prices_df.join(pd.get_dummies(prices_df["GPU_BRAND"], prefix="GPU_BRAND", dtype="int"))
prices_df = prices_df.join(pd.get_dummies(prices_df["OpSys"], prefix="OpSys", dtype="int"))
prices_df = prices_df.join(pd.get_dummies(prices_df["Memory_Type"], prefix="Memory_Type", dtype="int"))

In [0]:
prices_df = prices_df.drop(["Company", "TypeName", "ScreenResolution", "Cpu", "Memory", "Gpu", "OpSys", "GPU_BRAND", "Memory_Type", "CPU_BRAND"], axis=1)

In [0]:
prices_df